# Azure AI Video Retrieval APIs

To use the Video Retrieval APIs in a typical pattern, you'll follow these steps:

1. Create an index using PUT - Create an index.
2. Add video documents to the index using PUT - CreateIngestion.
3. Wait for the ingestion to complete, checking with GET - ListIngestions.
4. Search for a keyword or phrase using POST - SearchByText.

## Load Azure Configuration

In [26]:
import os
azure_computer_vision_endpoint = os.environ["AZURE_COMPUTER_VISION_ENDPOINT"]
azure_computer_vision_key = os.environ["AZURE_COMPUTER_VISION_KEY"]
VIDEO_INDEX = "my-video-index-01"

## Step 1 Create Index

In [27]:
import requests
import json

# Define the endpoint URL and subscription key
index_name = VIDEO_INDEX
endpoint_url = f"{azure_computer_vision_endpoint}/computervision/retrieval/indexes/{index_name}?api-version=2023-05-01-preview"
subscription_key = azure_computer_vision_key

# Define the headers
headers = {
    "Ocp-Apim-Subscription-Key": subscription_key,
    "Content-Type": "application/json"
}

# Define the payload
payload = {
    "metadataSchema": {
        # Define the fields that you want to include in the index
        "fields": [
            {
                "name": "cameraId",
                "searchable": False,
                "filterable": True,
                "type": "string"
            },
            {
                "name": "timestamp",
                "searchable": False,
                "filterable": True,
                "type": "datetime"
            }
        ]
    },
    "features": [
        # Define the features that you want to include in the index
        # https://learn.microsoft.com/en-us/azure/ai-services/computer-vision/reference-video-search#featuremodel
        {
            "name": "vision",
            "domain": "surveillance"
        },
        {
            "name": "speech"
        }
    ]
}

# Make the PUT request
response = requests.put(endpoint_url, headers=headers, json=payload)

# Print the response
print(response.status_code)
# Pretty-print the response JSON if the request was successful
print(json.dumps(response.json(), indent=4))

201
{
    "name": "my-video-index-01",
    "metadataSchema": {
        "language": "en",
        "fields": [
            {
                "name": "cameraid",
                "searchable": false,
                "filterable": true,
                "type": "string"
            },
            {
                "name": "timestamp",
                "searchable": false,
                "filterable": true,
                "type": "datetime"
            }
        ]
    },
    "userData": {},
    "features": [
        {
            "name": "vision",
            "modelVersion": "2023-05-31",
            "domain": "surveillance"
        },
        {
            "name": "speech",
            "modelVersion": "2023-06-30",
            "domain": "generic"
        }
    ],
    "eTag": "\"937ea37ab4c346b7a666ca5e1b487f3b\"",
    "createdDateTime": "2025-03-16T23:36:04.5126957Z",
    "lastModifiedDateTime": "2025-03-16T23:36:04.5126957Z"
}


## Step 2: Add video files to the index

In [28]:
import requests
import json

# Define the endpoint URL and subscription key
index_name = VIDEO_INDEX
endpoint_url = f"{azure_computer_vision_endpoint}/computervision/retrieval/indexes/{index_name}/ingestions/my-ingestion?api-version=2023-05-01-preview"
subscription_key = azure_computer_vision_key

headers = {
    "Ocp-Apim-Subscription-Key": subscription_key,
    "Content-Type": "application/json"
}

payload = {
    # Define the videos that you want to ingest
    "videos": [
    {
      "mode": "add",
      "documentId": "001",
      "documentUrl": "https://ziggystorage01.blob.core.windows.net/videos/dow.mp4",
      "metadata": {
        "cameraId": "camera1",
        "timestamp": "2025-03-07 16:40:33"
      }
    },
    {
      "mode": "add",
      "documentId": "002",
      "documentUrl": "https://ziggystorage01.blob.core.windows.net/videos/mercedes.mp4",
      "metadata": {
        "cameraId": "camera2",
        "timestamp": "2025-03-07 17:25:12"
      }
    }
  ]
}

response = requests.put(endpoint_url, headers=headers, json=payload)
print(response.status_code)
# Pretty-print the response JSON if the request was successful
print(json.dumps(response.json(), indent=4))

202
{
    "name": "my-ingestion",
    "state": "Running",
    "batchName": "96dbd9f3-003b-4815-aa82-58c292bb384c",
    "createdDateTime": "2025-03-16T23:37:13.1849604Z",
    "lastModifiedDateTime": "2025-03-16T23:37:13.7630850Z"
}


## Step 3: Wait for ingestion to complete

Wait for the status to be completed

In [30]:
import requests
import json

# Define the endpoint URL and subscription key
index_name = VIDEO_INDEX
endpoint_url = f"{azure_computer_vision_endpoint}/computervision/retrieval/indexes/{index_name}/ingestions?api-version=2023-05-01-preview&$top=20"
subscription_key = azure_computer_vision_key

headers = {
    "Ocp-Apim-Subscription-Key": subscription_key
}

# Make the GET request
response = requests.get(endpoint_url, headers=headers)

# Check the response status code
if response.status_code == 200:
    # Pretty-print the response JSON if the request was successful
    print(json.dumps(response.json(), indent=4))
else:
    # Print the error message if the request failed
    print(f"Error: {response.status_code}")
    print(response.text)

{
    "value": [
        {
            "name": "my-ingestion",
            "state": "Completed",
            "batchName": "96dbd9f3-003b-4815-aa82-58c292bb384c",
            "createdDateTime": "2025-03-16T23:37:13.1849604Z",
            "lastModifiedDateTime": "2025-03-16T23:38:15.0288387Z"
        }
    ]
}


## [Optional] List Documents

As an optional step, we can list the documents ingested

In [31]:
import requests
import json

# Define the endpoint URL and subscription key
index_name = VIDEO_INDEX
endpoint_url = f"{azure_computer_vision_endpoint}/computervision/retrieval/indexes/{index_name}/documents?api-version=2023-05-01-preview"
subscription_key = azure_computer_vision_key

headers = {
    "Ocp-Apim-Subscription-Key": subscription_key
}

# Make the GET request
response = requests.get(endpoint_url, headers=headers)

# Check the response status code
if response.status_code == 200:
    # Pretty-print the response JSON if the request was successful
    print(json.dumps(response.json(), indent=4))
else:
    # Print the error message if the request failed
    print(f"Error: {response.status_code}")
    print(response.text)

{
    "value": [
        {
            "documentId": "001",
            "documentUrl": "https://ziggystorage01.blob.core.windows.net/videos/dow.mp4",
            "metadata": {
                "cameraid": "camera1",
                "timestamp": "2025-03-07 16:40:33"
            },
            "createdDateTime": "2025-03-16T23:37:13.2007369Z",
            "lastModifiedDateTime": "2025-03-16T23:37:13.2007369Z",
            "userData": {}
        },
        {
            "documentId": "002",
            "documentUrl": "https://ziggystorage01.blob.core.windows.net/videos/mercedes.mp4",
            "metadata": {
                "cameraid": "camera2",
                "timestamp": "2025-03-07 17:25:12"
            },
            "createdDateTime": "2025-03-16T23:37:13.2007369Z",
            "lastModifiedDateTime": "2025-03-16T23:37:13.2007369Z",
            "userData": {}
        }
    ]
}


## Step 4: Perform searches with metadata - Vision Feature

In [32]:
import requests
import json

# Define the endpoint URL and subscription key
index_name = VIDEO_INDEX
endpoint_url = f"{azure_computer_vision_endpoint}/computervision/retrieval/indexes/{index_name}:queryByText?api-version=2023-05-01-preview"
subscription_key = azure_computer_vision_key

headers = {
    "Ocp-Apim-Subscription-Key": subscription_key,
    "Content-Type": "application/json"
}

# Define the payload
payload = {
  "queryText": "mercedes benz",
  "filters": {
    "stringFilters": [
      {
        "fieldName": "cameraId",
        "values": ["camera1","camera2"]
      }
    ],
    "featureFilters": ["vision"]
  }
}

# Make the POST request
response = requests.post(endpoint_url, headers=headers, json=payload)

# Check the response status code
if response.status_code == 200:
    # Pretty-print the response JSON if the request was successful
    print(json.dumps(response.json(), indent=4))
else:
    # Print the error message if the request failed
    print(f"Error: {response.status_code}")
    print(response.text)

{
    "value": [
        {
            "documentId": "002",
            "documentKind": "VideoInterval",
            "start": "00:00:10.0100000",
            "end": "00:00:19.0190000",
            "best": "00:00:15.0150000",
            "relevance": 0.32658258080482483
        },
        {
            "documentId": "002",
            "documentKind": "VideoInterval",
            "start": "00:00:19.0190000",
            "end": "00:00:23.0230000",
            "best": "00:00:22.0220000",
            "relevance": 0.3171845078468323
        },
        {
            "documentId": "002",
            "documentKind": "VideoInterval",
            "start": "00:00:37.0040000",
            "end": "00:00:42.0090000",
            "best": "00:00:38.0050000",
            "relevance": 0.3135458827018738
        },
        {
            "documentId": "002",
            "documentKind": "VideoInterval",
            "start": "00:00:52.0190000",
            "end": "00:00:59.0260000",
            "best": "00:0

## Step 4: Perform searches with metadata - Speech Feature

In [33]:
import requests
import json

# Define the endpoint URL and subscription key
index_name = VIDEO_INDEX
endpoint_url = f"{azure_computer_vision_endpoint}/computervision/retrieval/indexes/{index_name}:queryByText?api-version=2023-05-01-preview"
subscription_key = azure_computer_vision_key

headers = {
    "Ocp-Apim-Subscription-Key": subscription_key,
    "Content-Type": "application/json"
}

# Define the payload
payload = {
  "queryText": "leak",
  "dedup": False,
  "filters": {
    "stringFilters": [
      {
        "fieldName": "cameraId",
        "values": ["camera1","camera2"]
      }
    ],
    "featureFilters": ["speech"]
  }
}

# Make the POST request
response = requests.post(endpoint_url, headers=headers, json=payload)

# Check the response status code
if response.status_code == 200:
    # Pretty-print the response JSON if the request was successful
    print(json.dumps(response.json(), indent=4))
else:
    # Print the error message if the request failed
    print(f"Error: {response.status_code}")
    print(response.text)

{
    "value": [
        {
            "documentId": "001",
            "documentKind": "SpeechTextInterval",
            "start": "00:00:25.1200000",
            "end": "00:00:40.2400000",
            "best": "00:00:26.9200000",
            "relevance": 0.8906522989273071
        },
        {
            "documentId": "001",
            "documentKind": "SpeechTextInterval",
            "start": "00:00:46.2800000",
            "end": "00:00:54.0800000",
            "best": "00:00:46.2800000",
            "relevance": 0.8716200590133667
        },
        {
            "documentId": "001",
            "documentKind": "SpeechTextInterval",
            "start": "00:00:54.6400000",
            "end": "00:00:58.8400000",
            "best": "00:00:58.3600000",
            "relevance": 0.8193516731262207
        },
        {
            "documentId": "002",
            "documentKind": "SpeechTextInterval",
            "start": "00:00:22.2400000",
            "end": "00:00:29.4800000",
      

## Step 5: Delete the Index

In [34]:
import requests

# Define the endpoint URL and subscription key
index_name = VIDEO_INDEX
endpoint_url = f"{azure_computer_vision_endpoint}/computervision/retrieval/indexes/{index_name}?api-version=2023-05-01-preview"
subscription_key = azure_computer_vision_key

headers = {
    "Ocp-Apim-Subscription-Key": subscription_key
}

# Make the DELETE request
response = requests.delete(endpoint_url, headers=headers)

# Check the response status code
if response.status_code == 204:
    print("Index deleted successfully.")
else:
    print(f"Error: {response.status_code}")
    print(response.text)

Index deleted successfully.
